In [1]:
from VariationalBayes import ScalarParam, ModelParamsDict, VectorParam
from VariationalBayes.NormalParams import MVNParam, UVNParam
from VariationalBayes.GammaParams import GammaParam

from autograd import grad, hessian, jacobian, hessian_vector_product
import autograd.numpy as np

import VariationalBayes as vb


def ApproxEq(v1, v2, tol=1e-8):
    return np.max(np.abs(v1 - v2)) < tol

In [2]:
vec = np.array([0.5, 2.5])
for lb in [ -float("inf"), 0.2]:
    for ub in [ float("inf"), 2.7]:
        vec_free = vb.Parameters.unconstrain_vector(vec, lb=lb, ub=ub)
        vec_test = vb.Parameters.constrain(vec_free, lb=lb, ub=ub)
        assert ApproxEq(vec, vec_test)

In [3]:
mat = np.linalg.cholesky(np.full(4, 0.2).reshape(2, 2) + np.eye(2))
vec = vb.Parameters.VectorizeLDMatrix(mat)
assert ApproxEq(mat, vb.Parameters.UnvectorizeLDMatrix(vec))


In [4]:
mat = np.full(4, 0.2).reshape(2, 2) + np.eye(2)
vec = vb.Parameters.VectorizeLDMatrix(mat)

UnvecJac = jacobian(vb.Parameters.UnvectorizeLDMatrix)
print UnvecJac(vec)

VecJac = jacobian(vb.Parameters.VectorizeLDMatrix)
print VecJac(mat)


[[[ 1.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  1.  0.]
  [ 0.  0.  1.]]]
[[[ 1.  0.]
  [ 0.  0.]]

 [[ 0.  0.]
  [ 1.  0.]]

 [[ 0.  0.]
  [ 0.  1.]]]


In [5]:
mat = np.full(4, 0.2).reshape(2, 2) + np.eye(2)
vec = vb.Parameters.pack_posdef_matrix(mat)
assert ApproxEq(vb.Parameters.unpack_posdef_matrix(vec), mat)


In [6]:
pd_par = vb.PosDefMatrixParam('test', 2)
mat = np.full(4, 0.2).reshape(2, 2) + np.eye(2)

pd_par.set(mat)
vec = pd_par.get_free()
pd_par.set(np.eye(2))
pd_par.set_free(vec)

assert ApproxEq(pd_par.get(), mat)


In [7]:
# Test parameters
mat = np.full(4, 0.2).reshape(2, 2) + np.eye(2)

mvn_par = MVNParam('test', 2)
mvn_par.mean.set(np.array([1., 2.]))
mvn_par.cov.set(mat)

def TestMeanFun(new_mean):
    mvn_par.mean.set(new_mean)
    mean = mvn_par.e()
    return np.dot(mean.T, mean)

new_mean = np.array([4., 1.])
TestMeanFun(new_mean)
TestMeanFunGrad = grad(TestMeanFun)
print TestMeanFunGrad(new_mean)

def TestCovFun(new_cov):
    mvn_par.cov.set(new_cov)
    e_outer = mvn_par.e_outer()
    return np.trace(e_outer)

new_cov = np.eye(2) * 0.2
TestCovFun(new_cov)
TestCovFunGrad = grad(TestCovFun)
print TestCovFunGrad(new_cov)

mvn_free = mvn_par.get_free()

def TestCovFun(mvn_cov):
    return np.trace(mvn_cov)

Grad = grad(TestCovFun)
Grad(mvn_par.cov.get())

import copy
mvn_par_local = copy.deepcopy(mvn_par)
def TestMeanFunFree(mvn_free, mvn_par_local):
    mvn_par_local.set_free(mvn_free)
    e_outer = mvn_par_local.e_outer()
    return np.trace(e_outer)

# TestMeanFunFree(mvn_free, mvn_par_local)
TestMeanFunFreeGrad = grad(TestMeanFunFree)
TestMeanFunFreeGrad(mvn_free, mvn_par_local)


[ 8.  2.]
[[ 1.  0.]
 [ 0.  1.]]


array([ 8.        ,  2.        ,  0.89442719,  0.        ,  0.89442719])

In [16]:
par_dict = ModelParamsDict()
par_dict.push_param(MVNParam('mvn', 2))
mvn_mean = np.array([1., 2.])
par_dict['mvn'].mean.set(mvn_mean)
par_dict['mvn'].cov.set(mat)

par_free = par_dict.get_free()
par_dict['mvn'].mean.set(np.zeros(2))
par_dict['mvn'].cov.set(np.eye(2))
par_dict.set_free(par_free)
assert ApproxEq(par_dict['mvn'].mean.get(), mvn_mean)
assert ApproxEq(par_dict['mvn'].cov.get(), mat)

par_dict_copy = copy.deepcopy(par_dict)
def TestSetFun(par_free):
    par_dict_copy.set_free(par_free)
    return np.trace(par_dict_copy['mvn'].e_outer())

TestSetFunGrad = grad(TestSetFun)
TestSetFunGrad(par_free)

array([ 2.        ,  4.        ,  2.19089023,  0.36514837,  2.1602469 ])

In [ ]:

def DiGamma(x):
    return scipy.special.digamma(x)